In [ ]:
# Attempting to run code on a jupyter notebook (Failed)
from src import src_file

config_path="../src/myconfig.cfg"
verbose=True

parser = src_file.ConfigParser(config_path=config_path, verbose=verbose)

feat_extract = src_file.FeatureExtraction(method="SIFT", verbose=verbose)

# Extracting Features from Image Map
features = feat_extract.extract_features(parser.config_dict["image_map"], type="image")

# Extracting Features from video files (Only One Video Accepted)
features_video = feat_extract.extract_features(parser.config_dict["videos"], type="video")
features.extend(features_video)

parser.save_features(features)

In [ ]:
# Code for testing the Feature Extraction Module

if __name__ == '__main__':
    cmd_args = parse_args() #Read arguments passed on the command line
    parser = ConfigParser(config_path = cmd_args.config_file_path, verbose=cmd_args.verbose)
    
    feat_extract = FeatureExtraction(method="SIFT", verbose=cmd_args.verbose)
    
    # Extracting Features from Image Map
    features = feat_extract.extract_features(parser.config_dict["image_map"], type="image")
    
    # Extracting Features from video files (Only One Video Accepted)
    features_video = feat_extract.extract_features(parser.config_dict["videos"], type="video")
    features.extend(features_video)
    
    parser.save_features(features)

In [ ]:
# Code for testing the Feature Matching Module after Extracting Features

if __name__ == '__main__':
    cmd_args = parse_args() #Read arguments passed on the command line
    parser = ConfigParser(config_path = cmd_args.config_file_path, verbose=cmd_args.verbose)
    
    # Loading extracted features
    features = parser.load_features()
    
    # Matching Features
    feat_match = FeatureMatching(lib="sklearn", verbose=cmd_args.verbose)
    pts_in_map, pts_in_frame = feat_match.match_features(features)
    
    print(len(pts_in_map), len(pts_in_frame))